In [1]:
reset()

%display typeset

AA.<x,y>=PolynomialRing(QQ,'x,y')

K.<k> = FractionField(PolynomialRing(QQ, 'k'))
R.<x,y> = PolynomialRing(K,['x','y'])


######################################################
######################################################
######################################################

#@cached_function

def KS (p,q,r):
    if q>0: return (x-r)*(y-r) * KS(p-1,q-1,r+1)
    else: 
        expr=R(0)
        for a in [0..p]:
            Interim1=product(((k+1-s)/(s+1)) for s in (0..p-a-1))*product(((k+1-s)/(s+1)) for s in (0..a-1))
            Interim2=product(((k+1-s)/(s+1)) for s in (0..p-1))
            Interim3=product((x+k+1-a-r-i) for i in (0..p-a-1))*product((y-r-j) for j in (0..a-1))
            expr=expr+SR((Interim1/Interim2)*Interim3)
    return(expr)


#@cached_function

def KnSa (pp,qq): 
    return R(KS(pp,qq,0))



def RegularPart(polyn,param):
    polys=(SR(polyn).subs(k=k+param))
    polySing=(k*polys).expand().simplify_rational().subs(k=0)
    polyRegs=(polys-(1/k)*polySing).simplify_rational()
    
    polyReg=polyRegs.subs(k=0)
    return polyReg    

######################################################
######################################################
######################################################

def SingularPart(polyn,param):
    polys=(SR(polyn).subs(k=k+param))
    polySing=(k*polys).expand().simplify_rational().subs(k=0)
    return(polySing)




##################################################################
#
# Rks(la1,la2,kparam) computes the regular part of the 
# Knop-Sahi polynomial KnSa(la1,la2) at k=kparam.
#
##################################################################


def Rks(la1,la2,kparam):
    return SR(RegularPart(KnSa(la1,la2),kparam))



#######################################################################
#Determining the partition type; singular, quasi-regular, regular
#######################################################################

def PType (la1,la2,param):
    if (la1<=param) or (la1-la2)==(param+1) or (la1-la2)>=(2*param+3): 
        return 'irr'
    if (la1-la2)<=param and la1>=(param+1):
        return 'qreg'
    if (la1-la2)>=(param+2) and (la1-la2)<=(2*param+2):
        return 'sing' 




##############################################################
# NumPar(d) is the number of partiions lambda=(\lambda_1,\lambda_2) 
# of size at most d. 
##############################################################


def NumPar(d):
    if mod(d,2)==0:
        return ((d/2)+1)^2
    if mod(d,2)==1:
        return((d+1)*(d+3)/4)

    
    

    
#######################################
#   Calculation of \square f:
#######################################
    
def NAbla(ff):
    return (ff.derivative(x)-ff.derivative(y))/(4*(x-y))

def FuncValues (ff,param,mylevel):
    Dff=NAbla(ff)
    for ppsize in [0..mylevel]:
        for mu2 in [0..ppsize]:
            mu1=ppsize-mu2
            if (mu1>=mu2):
                if PType(mu1,mu2,param)=='irr':
                    print mu1,mu2, PType(mu1,mu2,param), (ff.subs(x=mu1-param-1,y=mu2))
                if PType(mu1,mu2,param)=='sing':
                    print mu1,mu2, PType(mu1,mu2,param), (ff.subs(x=mu1-param-1,y=mu2))
                if PType(mu1,mu2,param)=='qreg':
                    print mu1,mu2, PType(mu1,mu2,param), (Dff.subs(x=mu1-param-1,y=mu2))    
    
    

    
##############################################################
# (k)=x(x-1)...(x-k+1)/k! and FY(k)=y(y-1)...(y-k+1)/k!
##############################################################
def FX(k):
    if k>0:
        return (product((x-i+1) for i in [1..k])/factorial(k)) 
    if k==0:
        return AA(1)
    if k<0: 
        return AA(0)
    
def FY(k):    
    if k>0:
        return (product((y-i+1) for i in [1..k])/factorial(k)) 
    if k==0:
        return AA(1)
    if k<0: 
        return AA(0)

##########################################
##
## XaYb returns the value of [((x)_a)((y)_b)-((x)_b)((y)_a)] / 2(x-y) at x=c1 and y=c2
##
##########################################

def XaYb(aa,bb,c1,c2):
    if aa<=bb:
        a=aa
        b=bb
        Esign=1
    else:
        a=bb
        b=aa
        Esign=-1
    summ=QQ(0)
    for i in [0..b-a-1]:
        for j in [0..b-a-1-i]:
            Val1=(Esign*((-1)^(b-a+i+j)))/binomial(b,a)
            Val2=product((b-a-i-r) for r in [1..j])
            Val3=product((b-a-r) for r in [0..j])
            Val4=binomial(a+i,a)*binomial(a+j,a)*QQ(FX(a+i)(x=c1))*QQ(FY(a+j)(y=c2))
            summ=summ+((Val1*Val2*Val4)/Val3)
    return (1/2)*summ



##################################################
# Me(la1,la2): The column of the coeff matrix that 
# is indexed by the partition (la1,la2) is obtained 
# by the given formula. The numbering of partitions 
# starts as 0,1,2,3,... 
##################################################

def mE(la1,la2):
    d=la1+la2
    if mod(d,2)==0:
        return ((d*(d+2)/4)+la2)
    else:
        return (((d+1)/2)^2+la2)    


    

    
##################################################
#
# The function mEnt Computes the entry of the matrix A. For a partition (la1,la2) 
# of 'irr' or 'sing' type (corresponding to the k-parameter value "param"), 
# the function mEnt returns the value of (x)_e1 (y)_e2 + x_(e2) y_(e1) at the point 
# (la1-param-1,la2). For a partition (la1,la2) of quasi-regular type, it returns
# the value of (dx-dy)/2(x-y)[(x)_e1 (y)_e2 + x_(e2) y_(e1)] at the same point.
# the function returns
#
##################################################
    
def mEnt(la1,la2,et1,et2,param):
    if PType(la1,la2,param)=='irr' or PType(la1,la2,param)=='sing':
        f=AA(FX(et1)*FY(et2)+FX(et2)*FY(et1))
        return f(x=la1-param-1,y=la2)
    else: 
        Vall1=QQ(0)
        for t in [1..et2]:
            signt=QQ(((-1)^(t-1))/t)
            Vt=QQ(XaYb(et2-t,et1,la1-param-1,la2))
            Vall1=Vall1+(signt*Vt)
        for t in [1..et1]:
            signt=QQ(((-1)^(t-1))/t)
            Vt=QQ(XaYb(et1-t,et2,la1-param-1,la2))
            Vall1=Vall1+(signt*Vt)
        return (1/2)*Vall1



#########################################################################
#  BigCapelliFactorial(d,k,l) computes the big Capelli polynomial 
#  corresponding to the partition (d+k+l+2,d) for the given values k,l.
#  The polynomial is written in the factorial powers (x)_p(y)_q=FX(p)*FY(q). 
#  The nonzero coefficients are (conjecturally) only for the monomials
# where p=d+i,q=d+j with i+j\leq k+l+2. The output of the function is an
# N x 3 matrix: the first two columns give the values of i and j, and 
# the third column gives the value of the coefficient of the factorial 
# monomial (x)_{p} (y)_{q} + x_{q} y_{p}.
#########################################################################

@cached_function

def BigCapelliFactorial(lowerd,kparam,lparam):
    
    kvalue=kparam
    lvalue=lparam

    lambda2=lowerd
    lambda1=kvalue+2+lvalue+lambda2



    dvalue=lambda1+lambda2

    A=matrix(QQ,NumPar(dvalue),NumPar(dvalue),0)
    B=matrix(QQ,NumPar(dvalue),1,0)
    C=matrix(QQ,NumPar(dvalue),1,0)    
    
    #########
    D=matrix(QQ,NumPar(kvalue+lvalue+2),1,0)
    #########

    ##################################################################################
    # Filling the vector B with the values corresponding to a quasi regular partition.
    ##################################################################################

    CounterB=0
    for Prtsize in [0..dvalue]:
        for jj in [0..floor(Prtsize/2)]:
            mu1=Prtsize-jj
            mu2=jj
            if (mu1==lambda1) and (mu2==lambda2):
                B[CounterB,0]=QQ(1)
            else: 
                B[CounterB,0]=QQ(0)
            CounterB=CounterB+1



    #################################################
    # Filling up the entries of A
    #################################################


    rowA=0
    for Prtsize in [0..dvalue]:
        for jj in [0..floor(Prtsize/2)]:
            mu1=Prtsize-jj
            mu2=jj
            colA=0
            for Prtsize1 in [0..dvalue]:
                for ll in [0..floor(Prtsize1/2)]:
                    eta1=Prtsize1-ll
                    eta2=ll
                    A[rowA,colA]=mEnt(mu1,mu2,eta1,eta2,kvalue)
                    colA=colA+1
            rowA=rowA+1

    ################################################################
    #
    # Solving the linear system and obtaining the coefficients of the interpolated polynomial
    #
    ################################################################



    #C=A.inverse()*B
    C=A.solve_right(B)
#     expr=0
#     CounterB=0
#     for Prtsize in [0..dvalue]:
#         for jj in [0..floor(Prtsize/2)]:
#             mu1=Prtsize-jj
#             mu2=jj
#             if C[CounterB,0]<>0:
#                 show(mu1,mu2,'-->',C[CounterB,0])
#             expr=expr+(QQ(C[CounterB,0])*(FX(mu1)*FY(mu2)+FY(mu1)*FX(mu2)))
#             CounterB=CounterB+1

    
    
    CounterB=0
    CounterD=0
    for Prtsize in [0..dvalue]:
        for jj in [0..floor(Prtsize/2)]:
            mu1=Prtsize-jj
            mu2=jj
            if ((mu2-lambda2)>=0) and ((mu1+mu2-2*lambda2)<=(kvalue+lvalue+2)):
                #D[CounterD,0]=mu1-lambda2
                #D[CounterD,1]=mu2-lambda2
                #print mu1-lambda2,mu2-lambda2
                D[CounterD,0]=C[CounterB,0]
                CounterD=CounterD+1
            CounterB=CounterB+1
    
    return(D)




################################################
# lparam-adapted regular part of the Knop-Sahi polynomial KS_lambda where 
# lambda=(lowerd+lparam+1+iparam,lowerd+lparam+1+jparam). Note that we should
# choose parameters satisfying:
#
#      iparam >= jparam    and    iparam+japaram <= kparam-lparam
#
# The output is a matrix with NumPar(kparam+lparam+2) rows, containing
# coefficients of (x)_(d+a)y_(d+b) + (x)_(d+b)y_(d+a) for all possible a>=b.
#
#  NOTE: when iparam>=jparam>=0, we obtain the coefficients of the regular
#  part of the above KS polynomial (actually, the polynomial has no poles!) but
#  when iparam=kparam+1 and jparam=-lparam-1, we obtain the regular part of 
# KS(lowerd+kparam+lparam+2,lowerd); this polynomial has poles in its coefficients.
#
################################################

@cached_function

def RegKSfactorial(lowerd,kparam,lparam,iparam,jparam):
    
    B=matrix(QQ,NumPar(kparam+lparam+2),1,0)
    if (jparam>=0):
        dd=lowerd+lparam+jparam+1
        mm=iparam-jparam
        la1=dd+mm
        la2=dd
        CounterB=0
        for Prtsize in [0..kparam+lparam+2]:
            for jj in [0..floor(Prtsize/2)]:
                mu1=Prtsize-jj
                mu2=jj
                a1=mu1-lparam-jparam-1
                b1=mu2-lparam-jparam-1
                if (b1>=0) and (a1+b1<=mm):
                    Interim1=product(kparam+1-r for r in [0..mm-a1-1])
                    Interim2=product(kparam+1-r for r in [0..mm-b1-1])
                    Interim3=factorial(mm)*(factorial(dd)^2)*binomial(dd+a1,dd)*binomial(dd+b1,dd)
                    Interim4=factorial(mm-a1-b1)*product(kparam+1-r for r in [0..mm-1])
                    Interim5=1
                    if (mu1==mu2):
                        Interim5=QQ(1/2)
                    B[CounterB,0]=((Interim1*Interim2*Interim3*Interim5)/Interim4)
                CounterB=CounterB+1
        return (B)
    
    if (jparam==-lparam-1):
        mm=kparam+lparam+2
        CounterB=0
        for Prtsize in [0..mm]:
            for jj in [0..floor(Prtsize/2)]:
                mu1=Prtsize-jj
                mu2=jj
#                 if ((mm-mu1)<=kparam+1) and ((mm-mu2)>=kparam+2):
#                     Interim1=product(kparam+1-r for r in [0..mm-mu1-1])
#                     Interim2=factorial(mm-mu2-kparam-2)*((-1)^(mu2))
#                     Interim3=factorial(mm-kparam-2)
#                     Interim4=factorial(mm-mu1-mu2)
#                     Interim5=factorial(mm)*(factorial(lowerd)^2)*binomial(lowerd+mu1,lowerd)*binomial(lowerd+mu2,lowerd)
#                     Interim6=1
#                     if (mu1==mu2):
#                         Interim6=QQ(1/2)
#                     B[CounterB,0]=Interim6*Interim5*((Interim1*Interim2)/(Interim3*Interim4))

                Interim1=product(k+1-r for r in [0..mm-mu1-1])
                Interim2=product(k+1-r for r in [0..mm-mu2-1])
                Interim3=product(k+1-r for r in [0..mm-1])
                Interim4=(Interim1*Interim2)/Interim3
                Interim5=QQ(RegularPart(Interim4,kparam))
                Interim6=(factorial(mm)/factorial(mm-mu1-mu2))
                Interim61=(factorial(lowerd)^2)*binomial(lowerd+mu1,lowerd)*binomial(lowerd+mu2,lowerd)
                Interim7=QQ(1)
                if (mu1==mu2):
                    Interim7=QQ(1/2)
                B[CounterB,0]=Interim5*Interim6*Interim61*Interim7
                CounterB=CounterB+1
        return(B)
        
        
        #############################
        #############################
        #############################
    
############################################################
#
# RegKSRev returns the polynomial whose coefficients are 
# generated by RegKSfactorial.
#
############################################################

def RegKSRev(lowerd,kparam,lparam,iparam,jparam):
    expr=0
    M=matrix(QQ,NumPar(kparam+lparam+2),1,0)
    M=RegKSfactorial(lowerd,kparam,lparam,iparam,jparam)
    CounterB=0
    for Prtsize in [0..kparam+lparam+2]:
        for jj in [0..floor(Prtsize/2)]:
            mu1=Prtsize-jj
            mu2=jj
            expr=expr+QQ(M[CounterB,0])*AA((FX(lowerd+mu1)*FY(lowerd+mu2))+(FX(lowerd+mu2)*FY(lowerd+mu1)))
            CounterB=CounterB+1

    return expr



############################################################
#
# BigCapelliRev returns the polynomial whose coefficients are 
# generated by BigCapelliFactorial.
#
############################################################


def BigCapelliRev(lowerd,kparam,lparam):
    expr=0
    M=matrix(QQ,NumPar(kparam+lparam+2),1,0)
    M=BigCapelliFactorial(lowerd,kparam,lparam)
    CounterB=0
    for Prtsize in [0..kparam+lparam+2]:
        for jj in [0..floor(Prtsize/2)]:
            mu1=Prtsize-jj
            mu2=jj
            expr=expr+QQ(M[CounterB,0])*AA(FX(lowerd+mu1)*FY(lowerd+mu2)+FX(lowerd+mu2)*FY(lowerd+mu1))
            CounterB=CounterB+1

    return expr



        
######################################
# SpecialCoef computes the mysterious 
# coefficient a^{k,l}_{k-l,0}(d) for the 
# case k=l. 
######################################


def SpecialCoef(myd,myk):
    Fla=matrix(QQ,NumPar(2*myk+2),1,0)
    Fla1=matrix(QQ,NumPar(2*myk+2),1,0)
    R1=matrix(QQ,NumPar(2*myk+2),1,0)
    R2=matrix(QQ,NumPar(2*myk+2),1,0)
    coeffi=QQ(0)
    
    Fla=factorial(myd)*factorial(myd+myk+1)*BigCapelliFactorial(myd,myk,myk)
    R1=RegKSfactorial(myd,myk,myk,myk+1,-myk-1)
    R2=RegKSfactorial(myd,myk,myk,0,0)
    Fla1=Fla-(factorial(myk+1)/factorial(2*myk+2))*R1
    if (R2[mE(myk+1,myk+1),0])<>0:
        coeffi=QQ(Fla1[mE(myk+1,myk+1),0]/R2[mE(myk+1,myk+1),0])
        coefi=coeffi*((-1)^(myk+1))*factorial(myk+1)*product(myd+r for r in [1..myk+1])
        return(coefi)
    else: 
        print Fla
        print R1
        print R2

        
######################################################
#
#
#  FV(f,mu1,mu2,k) returns the box evaluation of f=f(x,y)
#  at (x,y)=(mu1-k-1,mu2). Here the interpretation of the
# box evaluation depends on the parameter k (because it 
# involves regular, quasi-regular, singular partitions).  
#
######################################################
        
def FV(ff,mu1,mu2,param):
    Dff=NAbla(ff)
    if PType(mu1,mu2,param)=='irr':
        return(ff.subs(x=mu1-param-1,y=mu2))
    if PType(mu1,mu2,param)=='sing':
        return(ff.subs(x=mu1-param-1,y=mu2))
    if PType(mu1,mu2,param)=='qreg':
        return(Dff.subs(x=mu1-param-1,y=mu2))  
        

# def SCFast(myd,myk):
#     P1=SR(RegKSRev(myd,myk,myk,myk+1,-myk-1))
#     P2=SR(RegKSRev(myd,myk,myk,0,0))
#     mu1=myd+myk+1
#     mu2=myd+myk+1
#     c1=FV(P1,mu1,mu2,myk)
#     c2=FV(P2,mu1,mu2,myk)
#     coef=((-1)^myk)*product(myd+r for r in [1..myk+1])*c1/(c2*binomial(2*myk+2,myk+1))
#     return(coef)




In [58]:
############################################
#
# The code in this box computes the Knop-Sahi 
# polynomial at (D,0). This code was written
# for the purpose of Conjecture 1. But the 
# Conjecture 1 was proved by a simpler, more 
# conceptual method.
#
############################################



# def Xf(n):
#     return product(x-r for r in [0..n-1])

# def Yf(n):
#     return product(y-r for r in [0..n-1])

# def Xs(n,const):
#     return product(x-r-const for r in [0..n-1])

# def Conj(D,kparam):
#     kk=kparam
#     retexpr=AA(0)
#     for a in [0..kk+1]:
#         for r in [1..D]:
#             Interim1=binomial(D,a)*(-1)^(a+r+1)*(SR(Xf(a)).subs(x=kk+1))*factorial(r-1)/(SR(Xf(a)).subs(x=D-kk-2))
#             Interim2=(binomial(D-a,r)*Xs(D-a-r,a)*Yf(a))-(binomial(a,r)*Xs(D-a,a)*Yf(a-r))+(binomial(a,r)*Xs(a-r,D-a)*Yf(D-a))-(binomial(D-a,r)*Xs(a,D-a)*Yf(D-a-r))
#             retexpr=retexpr+(Interim1*Interim2)
#     return retexpr

# def Conjj(D,kparam):
#     kk=kparam
#     retexpr=AA(0)
#     for a in [0..kk+1]:
#         Interim1=binomial(D,a)*(-1)^(a)*(SR(Xf(a)).subs(x=kk+1))/(SR(Xf(a)).subs(x=D-kk-2))
#         Interim2=(Xs(D-a,a-kk-1)*Yf(a))+(Xs(a,D-a-kk-1)*Yf(D-a))
#         retexpr=retexpr+(Interim1*Interim2)
#     return retexpr
        



In [2]:
def ACoeff (myk,myl,myi,myj,myd):
    b=myj
    a=myk-myl-myi
    if (a+b>0):
        Intrm1=(-1)^(myl+a+b)*binomial(a,b)
        Intrm2=factorial(myl)*factorial(myk-myl-a-b)*a
        Intrm3=product(myl+1+r for r in [1..b])*product(myl+a+r for r in [1..b])
        return QQ(Intrm1/(Intrm2*Intrm3))
    
    if (a+b==0):
        Intrm1=-(1/factorial(myk-myl))*factorial(myl+1)*(1/factorial(2*myl+2))
        
        Intrm2=SR(NAbla(Rks(myd+(2*myl)+2,myd,myl))).simplify_rational()
        
        Intrm22=Intrm2.subs(x=myd+myl+1,y=myd)
        
        Intrm3=SR(NAbla(Rks(myd+myl+1,myd+myl+1,myl))).simplify_rational()
        
        Intrm33=(Intrm3).subs(x=myd+myl+1,y=myd)
        
        return QQ(Intrm1*Intrm22/Intrm33)
        
        


def TestConjecture (la1,la2,kparam):
    if PType(la1,la2,kparam)=='irr':
        ff=SR((1/QQ(SR(KnSa(la1,la2)).subs(x=la1-kparam-1,y=la2,k=kparam)))*KnSa(la1,la2)).subs(k=kparam)
    if PType(la1,la2,kparam)=='qreg':
        ff=SR((1/QQ(SR(NAbla(KnSa(la1,la2))).subs(x=la1-kparam-1,y=la2,k=kparam)))*KnSa(la1,la2)).subs(k=kparam)
    if PType(la1,la2,kparam)=='sing':
        d=la2
        k1=kparam
        l=la1-la2-k1-2
        Intrm1=(factorial(l+1)/factorial(k1+l+2))*RegularPart(KnSa(d+k1+l+2,d),k1)
        Intrm2=0
        for i in [0..k1-l]:
            for j in [0..min(i,k1-l-i)]:
                Intrm2=Intrm2+(ACoeff(k1,l,i,j,d)*(SR(KnSa(d+l+1+i,d+l+1+j)).subs(k=k1)))
        ff=(1/(factorial(d+l+1)*factorial(d)))*(Intrm1+Intrm2)
    FuncValues(ff,kparam,la1+la2)
                  
                  
            
                  

In [3]:
FuncValues(Rks(3,0,1),1,3)

0 0 irr 0
1 0 irr 0
2 0 irr 0
1 1 irr -6
3 0 sing 6
2 1 qreg -3/2


In [14]:
(SR(KnSa(3,0)).subs(x=3-k-1,y=0)).expand()

6

In [40]:
p1=KnSa(3,0)
show(p1)

x^3 + ((3*k + 3)/(k - 1))*x^2*y + ((3*k + 3)/(k - 1))*x*y^2 + y^3 + 3*k*x^2 + (6*k + 6)*x*y + 3*k*y^2 + (3*k^2 - 1)*x + (3*k^2 - 1)*y + k^3 - k

In [14]:
p2=SingularPart(KnSa(3,0),1)
show(p2)

6*x^2*y + 6*x*y^2

In [13]:
FuncValues(Rks(3,0,1),1,3)

0 0 irr 0
1 0 irr 0
2 0 irr 0
1 1 irr -6
3 0 sing 6
2 1 qreg -3/2


In [49]:
p3=SR(p1-(6/(k-1))*KnSa(2,1)).simplify_rational()
show(p3)

k^3 + 3*k*x^2 + x^3 + 3*(k + x)*y^2 + y^3 + (3*k^2 - 1)*x + (3*k^2 + 6*k*x + 3*x^2 - 1)*y - k

In [42]:
SR(p1-(1/(k-1))*SingularPart(KnSa(3,0),1)).simplify_rational()

k^3 + 3*k*x^2 + x^3 + 3*(k + x)*y^2 + y^3 + (3*k^2 - 1)*x + (3*k^2 + 6*(k + 1)*x + 3*x^2 - 1)*y - k

In [48]:
p4=SR(p3).subs(k=1)
show(p4)

ValueError: power::eval(): division by zero

In [46]:
FuncValues(p4,1,3)

0 0 irr 0
1 0 irr 0
2 0 irr 0
1 1 irr 0
3 0 sing 6
2 1 qreg 0


In [33]:
Rks(3,0,1)

x^3 + 3*(x + 1)*y^2 + y^3 + 3*x^2 + (3*x^2 + 12*x + 2)*y + 2*x

In [32]:
SR(p3).subs(x=1-k-1,y=1).expand()

0

In [24]:
SR(p1).subs(x=1-k-1,y=1).expand().simplify_rational()

0

In [25]:
SR(p2).subs(x=1-k-1,y=1).expand().simplify_rational()

6*k^2 - 6*k

In [26]:
KnSa(2,1)

x^2*y + x*y^2 + (k - 1)*x*y

In [28]:
SR(KnSa(2,1)).subs(x=1-k-1,y=1).expand()

0

In [43]:
KnSa(2,1)

x^2*y + x*y^2 + (k - 1)*x*y

In [104]:
kk=3

for i in [kk+2..(2*kk)+2]:
    ff=SR(SingularPart(KnSa(i,0),kk))
    gg=NAbla(ff)
    value=gg.subs(x=i-kk-1,y=0)
    show(i,value)
    

5 -30

6 -90

7 -420

8 -2520

In [3]:
NAbla(KnSa(3,0))

(-3/2/(k - 1))*x + (-3/2/(k - 1))*y - 3/2

In [8]:
FuncValues(SingularPart(KnSa(3,0),1),1,3)

0 0 irr 0
1 0 irr 0
2 0 irr 0
1 1 irr 0
3 0 sing 0
2 1 qreg -3/2


In [7]:
FuncValues(ff,1,3)

0 0 irr 3
1 0 irr 3/2
2 0 irr 0
1 1 irr 0
3 0 sing -3/2
2 1 qreg 0


In [11]:
ff=NAbla(SingularPart(KnSa(3,0),1))

In [ ]:
FuncValues()